In [2]:
using SplittingOrderConditions

In [2]:
s=8
p=5
@time begin    
    eqs = generate_equations(1,s, palindromic=true, with_brackets=false)
    for q=2:p
        merge!(eqs,generate_equations(q,s, palindromic=true, with_brackets=false)) 
    end
end    
eqs

  0.294642 seconds (577.99 k allocations: 39.528 MB, 1.11% gc time)


Dict{Array{Int64,1},AbstractString} with 8 entries:
  [0,1]       => "+2*b1*a2+2*b1*a3+2*b1*a4+2*b1*b4+2*b1*b3+2*b1*b2+2*b1*b1+2*b2…
  [0]         => "+1*a1+1*a2+1*a3+1*a4+1*b4+1*b3+1*b2+1*b1-1"
  [0,0,0,0,1] => "+5*b1*a2^4+20*b1*a2^3*a3+20*b1*a2^3*a4+20*b1*a2^3*b4+20*b1*a2…
  [0,0,1]     => "+3*b1*a2^2+6*b1*a2*a3+6*b1*a2*a4+6*b1*a2*b4+6*b1*a2*b3+6*b1*a…
  [0,0,1,0,1] => "+60*b1*a2*b2*a3^2+120*b1*a2*b2*a3*a4+120*b1*a2*b2*a3*b4+120*b…
  [0,0,0,1,1] => "+10*b1^2*a2^3+30*b1^2*a2^2*a3+30*b1^2*a2^2*a4+30*b1^2*a2^2*b4…
  [0,0,0,1]   => "+4*b1*a2^3+12*b1*a2^2*a3+12*b1*a2^2*a4+12*b1*a2^2*b4+12*b1*a2…
  [0,0,1,1]   => "+6*b1^2*a2^2+12*b1^2*a2*a3+12*b1^2*a2*a4+12*b1^2*a2*b4+12*b1^…

In [3]:
vars = ["a1", "a2", "a3", "a4", "b1", "b2", "b3", "b4"]
funs = Dict{Array{Int64,1},Function}()
for lw in keys(eqs)
    for k =1:length(vars)
        eqs[lw] = replace(eqs[lw], vars[k], string("x(",k,")"))        
    end
    funs[lw] = SplittingOrderConditions.compile_fg(eqs[lw], length(vars))
end    
funs=collect(values(funs))

# of attemps to call form = 306
# of attemps to call form = 11
# of attemps to call form = 331
# of attemps to call form = 108
# of attemps to call form = 555
# of attemps to call form = 221
# of attemps to call form = 219
# of attemps to call form = 331


8-element Array{Function,1}:
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)

In [4]:
a = [0.0948061248011184,0.295318061899783,1.53754156461765,-1.76438699139276,
     0.0815411353469756,-0.000346187375210155,0.516036508037932,0.239489784064514]
b = [0.239489784064514,0.516036508037932,-0.000346187375210155,0.0815411353469756,
    -1.76438699139276,1.53754156461765,0.295318061899783,0.0948061248011184]
x = [BigFloat(t) for t in vcat(a[1:4],b[1:4])]
n = length(funs)
J=zeros(BigFloat,n,n)
f=zeros(BigFloat,n)
g=zeros(BigFloat,n)
while true
    for k=1:n
        f[k] = funs[k](x,g)
        J[k,:] = g.'
    end
    res = maximum(abs(f))
    println("res = ", res)
    x -= J\f
    if res<eps(BigFloat)*100
        break
    end
end
a = vcat(x[1:4],reverse(x[5:8]))
b = reverse(a);

res = 3.284152125467809482767519642677476869662589806216058320636409027683003186193324e-14
res = 1.066634991358370061492688570360775527091885272177340293678612410515173065352047e-24
res = 6.520443917322062402922169467673567152184416441919793343723693167523111596720383e-48
res = 1.17451892349284446905254385334085434167177604956333426836322367194292848523009e-75


In [5]:
a

8-element Array{BigFloat,1}:
  9.480612480096383755810238332092734295342265842893021596275902029430768282124701e-02
  2.953180618997683333982799787452393320472145416348581713921729632789296702805809e-01
  1.537541564619954771866660112165026811200228013610318688766209981025206572878256    
 -1.764386991395374283204348438900170800972430834544106121236887781167578344571381    
  8.154113534711735851774025233958719758552038570752172710018665663195316858490895e-02
 -3.461873752067720546596206064633419451746355641025988198797095382700431672899566e-04
  5.16036508038544019149622640437979835447145581419329353984986889207173627040678e-01 
  2.394897840642327347686026924978736236840742893072505628504519802682776661329925e-01

In [6]:
b

8-element Array{BigFloat,1}:
  2.394897840642327347686026924978736236840742893072505628504519802682776661329925e-01
  5.16036508038544019149622640437979835447145581419329353984986889207173627040678e-01 
 -3.461873752067720546596206064633419451746355641025988198797095382700431672899566e-04
  8.154113534711735851774025233958719758552038570752172710018665663195316858490895e-02
 -1.764386991395374283204348438900170800972430834544106121236887781167578344571381    
  1.537541564619954771866660112165026811200228013610318688766209981025206572878256    
  2.953180618997683333982799787452393320472145416348581713921729632789296702805809e-01
  9.480612480096383755810238332092734295342265842893021596275902029430768282124701e-02

In [7]:
a=[0.201651044312325,0.562615975356585,0.253874038247542,-0.835351693190397,
    0.0680149460931738,-0.102733803148438,0.273128836056527,0.578800656272682]
b=[0.578800656272682,0.273128836056527,-0.102733803148438,0.0680149460931738,
    -0.835351693190397,0.253874038247542,0.562615975356585,0.201651044312325]
x = [BigFloat(t) for t in vcat(a[1:4],b[1:4])]
n = length(funs)
J=zeros(BigFloat,n,n)
f=zeros(BigFloat,n)
g=zeros(BigFloat,n)
while true
    for k=1:n
        f[k] = funs[k](x,g)
        J[k,:] = g.'
    end
    res = maximum(abs(f))
    println("res = ", res)
    x -= J\f
    if res<eps(BigFloat)*100
        break
    end
end
a = vcat(x[1:4],reverse(x[5:8]))
b = reverse(a);

res = 1.883271295730021542957273486643548024825501147245366326589017518861298133585104e-15
res = 3.007912408428557697270705224694414107343315480010314519125487119802583882403688e-27
res = 1.389569169748473132922779612923049495867301989226660858474497314955700950408356e-52
res = 3.972637535343444527677721856888183802713360167640689437110903596277552229454716e-76


In [8]:
a

8-element Array{BigFloat,1}:
  2.016510443123242295010717537127204350343897695135273956068179634356826878006001e-01
  5.626159753565692004726136871495739306245002418928744167752083427967419233021117e-01
  2.538740382475548454088261616327640537887272103604989852329201422299961638614295e-01
 -8.353516931903706357129392355396398222058695622004032927093969848250637070847104e-01
  6.801494609316509176653714245969736234055991715893380050062743095376070430288747e-02
 -1.027338031484321420816894557883886342801172074332317945637108267769652758423583e-01
  2.731288360565244789123179237944314885888137612222677858196072939859034375087217e-01
  5.788006562726649317332620225788411861089958694855327033379266381999440661513172e-01

In [9]:
b

8-element Array{BigFloat,1}:
  5.788006562726649317332620225788411861089958694855327033379266381999440661513172e-01
  2.731288360565244789123179237944314885888137612222677858196072939859034375087217e-01
 -1.027338031484321420816894557883886342801172074332317945637108267769652758423583e-01
  6.801494609316509176653714245969736234055991715893380050062743095376070430288747e-02
 -8.353516931903706357129392355396398222058695622004032927093969848250637070847104e-01
  2.538740382475548454088261616327640537887272103604989852329201422299961638614295e-01
  5.626159753565692004726136871495739306245002418928744167752083427967419233021117e-01
  2.016510443123242295010717537127204350343897695135273956068179634356826878006001e-01

In [10]:
a=Complex{Float64}[0.0616859578475632 - 0.0222823670735455im,0.100252119833815 + 0.0665813505646068im,
    0.173445604000608 + 0.0645347825330634im,0.142290226977369 - 0.0867529559497755im,
    0.108004395790385 - 0.107731532365639im,0.180508075049188 - 0.00844653859129045im,
    0.126231681726252 + 0.126845335227947im,0.107581938774818 - 0.0327480743453665im]
b=Complex{Float64}[0.107581938774818 - 0.0327480743453665im,
    0.126231681726252 + 0.126845335227947im,0.180508075049188 - 0.00844653859129045im,
    0.108004395790385 - 0.107731532365639im,0.142290226977369 - 0.0867529559497755im,
    0.173445604000608 + 0.0645347825330634im,0.100252119833815 + 0.0665813505646068im,
    0.0616859578475632 - 0.0222823670735455im]
x = [Complex{BigFloat}(t) for t in vcat(a[1:4],b[1:4])]
n = length(funs)
J=zeros(Complex{BigFloat},n,n)
f=zeros(Complex{BigFloat},n)
g=zeros(Complex{BigFloat},n)
while true
    for k=1:n
        f[k] = funs[k](x,g)
        J[k,:] = g.'
    end
    res = maximum(abs(f))
    println("res = ", res)
    x -= J\f
    if res<eps(BigFloat)*100
        break
    end
end
a = vcat(x[1:4],reverse(x[5:8]))
b = reverse(a);

res = 9.351088856389245945865075297989778198569056097910885365085548448543841526082178e-15
res = 7.83776066642287586649210479063798498249465108352910611202340891644682250701444e-22
res = 1.294048616391037085011696879277947599505724606761281449776548050674539418794095e-41
res = 2.286229959766329520787732464520840544202940250352902781442375028177954129952915e-77


In [11]:
a

8-element Array{Complex{BigFloat},1}:
 6.168595784859109424379843422161445371422367031536680311317629391969514199290263e-02-2.228236707396364035949975968671562744923709082096192480324603659193214509684778e-02im
  1.00252119839293154352277026785837311496548538536478455172567661832231658414216e-01+6.658135056707195071948508981148362921443577049056794133051610895433609171026112e-02im
  1.73445604001127437122682376753370831303599352140055084824863577685824958080055e-01+6.453478253014967114573942989676273618816291601334291727780863420463680225705505e-02im
 1.422902269732557020533341214022138791440669214243052322164718960977419693026926e-01-8.675295595502738947058641357262101986267295779085090526630706063607479973831568e-02im
 1.080043957830595334777308033784256203158581849001575504324231584293991298551569e-01-1.077315323540064128413870530898261896293948381271446508518411971977603831770945e-01im
 1.805080750470063893413572056580201364732210819809652874391233168248962808320333e-01-8.446538597

In [12]:
b

8-element Array{Complex{BigFloat},1}:
 1.075819387789741967547922811641949048475227121994847997955774510450355136737389e-01-3.274807434427748379662523726069659700735896579959728170754567288579861083246104e-02im
 1.262316817286924926540277506363228627049595385031867870057966441651753478492105e-01+1.268453352278416445318635499224149685985053572106080328320760098157251685786566e-01im
 1.805080750470063893413572056580201364732210819809652874391233168248962808320333e-01-8.446538597788339928989606020801900052440191175964128811460785663132123701252231e-03im
 1.080043957830595334777308033784256203158581849001575504324231584293991298551569e-01-1.077315323540064128413870530898261896293948381271446508518411971977603831770945e-01im
 1.422902269732557020533341214022138791440669214243052322164718960977419693026926e-01-8.675295595502738947058641357262101986267295779085090526630706063607479973831568e-02im
  1.73445604001127437122682376753370831303599352140055084824863577685824958080055e-01+6.453478253

In [6]:
p,s = 6,10
@time begin
    eqs = generate_equations(1,s, symmetric=true, with_brackets=false)
    for q=2:p
        merge!(eqs,generate_equations(q,s, symmetric=true, with_brackets=false)) 
    end
end    
eqs

  0.151683 seconds (1.40 M allocations: 277.828 MB, 18.30% gc time)


Dict{Array{Int64,1},AbstractString} with 10 entries:
  [0,0,0,0,1] => "+5*b1*a2^4+20*b1*a2^3*a3+20*b1*a2^3*a4+20*b1*a2^3*a5+20*b1*a2…
  [0]         => "+1*a1+1*a2+1*a3+1*a4+1*a5+1*a5+1*a4+1*a3+1*a2+1*a1-1"
  [0,0,1,0,1] => "+60*b1*a2*b2*a3^2+120*b1*a2*b2*a3*a4+120*b1*a2*b2*a3*a5+120*b…
  [0,0,1]     => "+3*b1*a2^2+6*b1*a2*a3+6*b1*a2*a4+6*b1*a2*a5+6*b1*a2*a5+6*b1*a…
  [0,1,1]     => "+3*b1^2*a2+3*b1^2*a3+3*b1^2*a4+3*b1^2*a5+3*b1^2*a5+3*b1^2*a4+…
  [0,0,1,1,1] => "+10*b1^3*a2^2+20*b1^3*a2*a3+20*b1^3*a2*a4+20*b1^3*a2*a5+20*b1…
  [0,1,0,1,1] => "+60*b1^2*a2*b2*a3+60*b1^2*a2*b2*a4+60*b1^2*a2*b2*a5+60*b1^2*a…
  [1]         => "+1*b1+1*b2+1*b3+1*b4+1*b5+1*b4+1*b3+1*b2+1*b1-1"
  [0,0,0,1,1] => "+10*b1^2*a2^3+30*b1^2*a2^2*a3+30*b1^2*a2^2*a4+30*b1^2*a2^2*a5…
  [0,1,1,1,1] => "+5*b1^4*a2+5*b1^4*a3+5*b1^4*a4+5*b1^4*a5+5*b1^4*a5+5*b1^4*a4+…

In [7]:
vars = ["a1", "a2", "a3", "a4", "a5", "b1", "b2", "b3", "b4", "b5"]
funs = Dict{Array{Int64,1},Function}()
for lw in keys(eqs)
    for k =1:length(vars)
        eqs[lw] = replace(eqs[lw], vars[k], string("x(",k,")"))        
    end
    funs[lw] = SplittingOrderConditions.compile_fg(eqs[lw], length(vars))
end    
funs=collect(values(funs))

# of attemps to call form = 17
# of attemps to call form = 76
# of attemps to call form = 40
# of attemps to call form = 4
# of attemps to call form = 96
# of attemps to call form = 117
# of attemps to call form = 111
# of attemps to call form = 31
# of attemps to call form = 180
# of attemps to call form = 13


10-element Array{Function,1}:
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)
 (anonymous function)

In [11]:
a=[0.0964689167981036,-0.0145982806525041,0.433353651607097,-0.116090392126529,0.100866104373833,
 0.100866104373833,-0.116090392126529,0.433353651607097,-0.0145982806525041,0.0964689167981036]
b=[0.489242960734905,-0.220019815637604,1.00660683634964,0.0228895038618238,-1.59743897061752,
 0.0228895038618238,1.00660683634964,-0.220019815637604,0.489242960734905,0.0]
x = [BigFloat(t) for t in vcat(a[1:5],b[1:5])]
n = length(funs)
J=zeros(BigFloat,n,n)
f=zeros(BigFloat,n)
g=zeros(BigFloat,n)
while true
    for k=1:n
        f[k] = funs[k](x,g)
        J[k,:] = g.'
    end
    res = maximum(abs(f))
    println("res = ", res)
    x -= J\f
    if res<eps(BigFloat)*100
        break
    end
end
a = vcat(x[1:5],reverse(x[1:5]))
b = vcat(x[6:10],reverse(x[6:9]),0.0);

res = 7.416297552265209282990706504998603905159536735130152163464162304058892004944007e-14
res = 5.384707198574727682366502226133004128954882850823408107108087565394904028632264e-25
res = 1.214673622528925105716984009612475662640673609695024820604880252467354833017309e-48
res = 2.659939914969088944618996373742523067903728112246374666587300668811926275374028e-75
res = 3.281744050935888957646813707864151837024080138485786926309007318664064885201722e-75
res = 4.421718300208355648197812153753804580411392186591376069132136176726319003219163e-75
res = 5.630781897921577895751901414545860520367632237612455463035454662549921855661902e-75
res = 3.039931331393244508135995855705740649032832128281571047528343621499344314713174e-75
res = 1.537238002806811143318770631578471123658648064869658086534219217690009340962912e-75


In [12]:
a

10-element Array{BigFloat,1}:
  9.646891679810619937443049275008358482835693429537284238528867472065886634103627e-02
 -1.459828065250176955465361762701549013102193630061814882523790364194776371933409e-02
  4.333536516070643994347343675719705675448315787381982840769106049886741848423145e-01
 -1.160903921266695967917320487000019282127929879247905837973885756024904701301485e-01
  1.008661043740007675372208060049632659706264111918376061604271995351051826661311e-01
  1.008661043740007675372208060049632659706264111918376061604271995351051826661311e-01
 -1.160903921266695967917320487000019282127929879247905837973885756024904701301485e-01
  4.333536516070643994347343675719705675448315787381982840769106049886741848423145e-01
 -1.459828065250176955465361762701549013102193630061814882523790364194776371933409e-02
  9.646891679810619937443049275008358482835693429537284238528867472065886634103627e-02

In [13]:
b

10-element Array{BigFloat,1}:
  4.892429607350192830623231681136870239270108776576337829030001377397791693577661e-01
 -2.20019815637705127687647540776716432200249988566297821664215478263040755655372e-01 
  1.006606836349979391618513959702873301121775486597600961900798819919415679345793    
  2.288950386174900383783989736121501493665299752131414671521149187897970934339244e-02
 -1.597438970618085101662058968802117815570378746420502139709589942550267604783183    
  2.288950386174900383783989736121501493665299752131414671521149187897970934339244e-02
  1.006606836349979391618513959702873301121775486597600961900798819919415679345793    
 -2.20019815637705127687647540776716432200249988566297821664215478263040755655372e-01 
  4.892429607350192830623231681136870239270108776576337829030001377397791693577661e-01
  0.000000000000000000000000000000000000000000000000000000000000000000000000000000    

In [14]:
a = [0.0951762545417729,-0.127950285523728,0.105972953453297,0.44822227660084,-0.0214211990721815,
    -0.0214211990721815,0.44822227660084,0.105972953453297,-0.127950285523728,0.0951762545417729]
b = [0.666296893997668,0.0246189009520825,-0.410725533617898,0.657729262050858,-0.875839046765421,
    0.657729262050858,-0.410725533617898,0.0246189009520825,0.666296893997668,0.0]
x = [BigFloat(t) for t in vcat(a[1:5],b[1:5])]
n = length(funs)
J=zeros(BigFloat,n,n)
f=zeros(BigFloat,n)
g=zeros(BigFloat,n)
while true
    for k=1:n
        f[k] = funs[k](x,g)
        J[k,:] = g.'
    end
    res = maximum(abs(f))
    println("res = ", res)
    x -= J\f
    if res<eps(BigFloat)*100
        break
    end
end
a = vcat(x[1:5],reverse(x[1:5]))
b = vcat(x[6:10],reverse(x[6:9]),0.0);

res = 7.209772448160544708039948105783960194803866394381537076400884259848746974205426e-15
res = 3.597776929000627609944718029003979026666995848234672936582448937693729617662281e-26
res = 7.313922674070829904595914062967864698719264199684888123211906062209938988262284e-50
res = 4.736398691934609474235327349754594139783931449870523072098937371920587067046894e-76


In [15]:
a

10-element Array{BigFloat,1}:
  9.517625454177405267746114335519341985760040729522568007850953110585358755374361e-02
 -1.279502855236867794121919162142941145226537625743049251429521253190326022796313e-01
  1.059729534532511314379358760871699825400810685169280417098192375774895373335818e-01
  4.482222766008274841685163418656120108361212993221011535809248353519807158017584e-01
 -2.142119907216588887172144509368129871114901255994995022630147871629123840945666e-02
 -2.142119907216588887172144509368129871114901255994995022630147871629123840945666e-02
  4.482222766008274841685163418656120108361212993221011535809248353519807158017584e-01
  1.059729534532511314379358760871699825400810685169280417098192375774895373335818e-01
 -1.279502855236867794121919162142941145226537625743049251429521253190326022796313e-01
  9.517625454177405267746114335519341985760040729522568007850953110585358755374361e-02

In [16]:
b

10-element Array{BigFloat,1}:
  6.662968939977078013420749890716806824071237024753835291923641586580294696593022e-01
  2.461890095210508713078430308713061735411857651176931675347343093620679145080255e-02
 -4.107255336179511323199287391819902473673343230370801516518815380001767269263703e-01
  6.577292620509131776893513000933904214483928344542936453566815536988930680071681e-01
 -8.758390467655498676845637061404229476846015808087326793012752105859052043818054e-01
  6.577292620509131776893513000933904214483928344542936453566815536988930680071681e-01
 -4.107255336179511323199287391819902473673343230370801516518815380001767269263703e-01
  2.461890095210508713078430308713061735411857651176931675347343093620679145080255e-02
  6.662968939977078013420749890716806824071237024753835291923641586580294696593022e-01
  0.000000000000000000000000000000000000000000000000000000000000000000000000000000    

In [18]:
a = Complex{Float64}[0.0499277424319449 - 0.0157522185631533im,0.0983001551320812 + 0.0388302144919185im,
    0.121039269110474 + 0.0605469125864282im,0.122393043066897 - 0.0432248096226892im,
    0.108339790258602 - 0.0404000988925042im,0.108339790258602 - 0.0404000988925042im,
    0.122393043066897 - 0.0432248096226892im,0.121039269110474 + 0.0605469125864282im,
    0.0983001551320812 + 0.0388302144919185im,0.0499277424319449 - 0.0157522185631533im]
b = Complex{Float64}[0.0998554848638346 - 0.0315044371263929im,0.0967448254005983 + 0.109164866110368im,
    0.145333712820249 + 0.0119289590622208im,0.0994523733132041 - 0.0983785783075674im,
    0.117227207204229 + 0.0175783805227438im,0.0994523733132041 - 0.0983785783075674im,
    0.145333712820249 + 0.0119289590622208im,0.0967448254005983 + 0.109164866110368im,
    0.0998554848638346 - 0.0315044371263929im,0.0 + 0.0im]
x = [Complex{BigFloat}(t) for t in vcat(a[1:5],b[1:5])]
n = length(funs)
J=zeros(Complex{BigFloat},n,n)
f=zeros(Complex{BigFloat},n)
g=zeros(Complex{BigFloat},n)
while true
    for k=1:n
        f[k] = funs[k](x,g)
        J[k,:] = g.'
    end
    res = maximum(abs(f))
    println("res = ", res)
    x -= J\f
    if res<eps(BigFloat)*100
        break
    end
end
a = vcat(x[1:5],reverse(x[1:5]))
b = vcat(x[6:10],reverse(x[6:9]),0.0);

res = 6.888555542711299086476943872169627753071719015316292411091017493446833253270753e-15
res = 2.67214317726335310277843661602876726389504646429865219002437283245314349535097e-25
res = 6.361520762561542461267928877680738112845955222578785149062069581536276013868953e-25
res = 1.590380190618021414143697657972164164620559756208048393079079543328794516878473e-25
res = 3.975950476542033634838688380128249280421134784465551723705694397953339493644456e-26
res = 9.939876191351309211446387285009475855803350068307061936752242196253891853129001e-27
res = 2.484969047837355443405314870405189472658804595298545380348125461358124289278937e-27
res = 6.212422619592798784192940835776947021198045843857435542172587480536846476811326e-28
res = 1.553105654898125968008192297559917656413519467708070513941423019213422135444296e-28
res = 3.882764137245222759970427342885135414154826258762833450936503737992165453559948e-29
res = 9.70691034311294169986350175482935271870763740414677154148594339454724761417328e-30
r

In [19]:
a

10-element Array{Complex{BigFloat},1}:
  4.992774243207751233140316737345405552906116945772420601396375528518055836256615e-02-1.57522185631781869479764316880164898750687820907734932750568211538058603426824e-02im
 9.830015513226510526936792773770740492706220824086302856932014407652809307039369e-02+3.883021449205753234327164786271250390299916316723952918461789556537388390027532e-02im
 1.210392691099798367617760545459801626463452719622628887874761873776470270747108e-01+6.054691258661095631720159537904939015820614268940922954568680966232944993535452e-02im
 1.223930430662852233577811745806538380516644109365183881574061509986863969016695e-01-4.322480962238632773777774102166024159922863467978430331289947599204891110224985e-02im
 1.083397902593923222796716757622045388458669394026314884718337622619579245906598e-01-4.040009889310397397471907053208516258690788908609096214234840808184856239069733e-02im
 1.083397902593923222796716757622045388458669394026314884718337622619579245906598e-01-4.04000988

In [20]:
b

10-element Array{Complex{BigFloat},1}:
 9.985548486415502466280633474690811105409056781826007208470273781488381450771743e-02-3.150443712635637389595286337603297975743235489197843877830899416986778572905506e-02im                                                                              
 9.674482540037518587592952072850669882604908064628649275898113066600024962702513e-02+1.091648661104714385824961591014579875823867461763163994379319339403454866013102e-01im                                                                              
 1.453337128195844876476225883634536264791858755743990134373810436065370528617955e-01+1.192895906275047405190703165664079269370183241550006936333121931783407612192221e-02im                                                                              
 9.945237331298595906793976079785404954190064214309577981380471874317768408407046e-02-9.837857830752312952746251369996127591153024579862922480843080135070461771609553e-02im                                    